# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

Прочитаем таблицу ('/datasets/data.csv') и запишем в переменную df, выведем на экран первые 15 строк.

In [1]:
import pandas as pd
df = pd.read_csv('/datasets/data.csv')
df.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Отобразим общую информацию о датафрейме.

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Проведем предварительный анализ данных датафрейма.

Отобразим уникальные значения столбца children и подсчитаем их количество.

In [3]:
display(df['children'].sort_values().unique())
df['children'].value_counts()

array([-1,  0,  1,  2,  3,  4,  5, 20])

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Посчитаем нулевые значения в столбце days_employed.

In [4]:
df[df['days_employed'] == 0]['days_employed'].count()

0

Отобразим уникальные значения столбца education.

In [5]:
df['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

Отобразим уникальные значения в столбце dob_years и посчитаем нулевые значения.

In [6]:
print(sorted(df['dob_years'].unique()))
df[df['dob_years'] == 0]['dob_years'].count()

[0, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75]


101

Отобразим уникальные значения в столбце family_status.

In [7]:
sorted(df['family_status'].unique())

['Не женат / не замужем',
 'в разводе',
 'вдовец / вдова',
 'гражданский брак',
 'женат / замужем']

Отобразим уникальные значения в столбце gender посчитаем их количество.

In [8]:
print(sorted(df['gender'].unique()))
df['gender'].value_counts()

['F', 'M', 'XNA']


F      14236
M       7288
XNA        1
Name: gender, dtype: int64

Отобразим уникальные значения в столбце income_type.

In [9]:
sorted(df['income_type'].unique())

['безработный',
 'в декрете',
 'госслужащий',
 'компаньон',
 'пенсионер',
 'предприниматель',
 'сотрудник',
 'студент']

Посчитаем нулевые значения в столбце total_income.

In [10]:
df[df['total_income'] == 0]['total_income'].count()

0

Отобразим уникальные значения в столбце purpose.

In [11]:
print(sorted(df['purpose'].unique()))

['автомобили', 'автомобиль', 'высшее образование', 'дополнительное образование', 'жилье', 'заняться высшим образованием', 'заняться образованием', 'на покупку автомобиля', 'на покупку подержанного автомобиля', 'на покупку своего автомобиля', 'на проведение свадьбы', 'недвижимость', 'образование', 'операции с жильем', 'операции с коммерческой недвижимостью', 'операции с недвижимостью', 'операции со своей недвижимостью', 'покупка жилой недвижимости', 'покупка жилья', 'покупка жилья для сдачи', 'покупка жилья для семьи', 'покупка коммерческой недвижимости', 'покупка недвижимости', 'покупка своего жилья', 'получение высшего образования', 'получение дополнительного образования', 'получение образования', 'приобретение автомобиля', 'профильное образование', 'ремонт жилью', 'свадьба', 'свой автомобиль', 'сделка с автомобилем', 'сделка с подержанным автомобилем', 'строительство жилой недвижимости', 'строительство недвижимости', 'строительство собственной недвижимости', 'сыграть свадьбу']


**Вывод**

Анализ таблицы показал: 
1. Датафрейм содержит 21525 строк и 12 столбцов, в них содержатся данные различных форматов: целочисленный, вещественный, строковый. 
2. В столбцах days_employed и total_income содержатся пропуски типа NaN и не содержится нулевых значений. Кроме этого в столбце "общий трудовой стаж в днях" содержатся артефакты: часть значений приведена с отрицательным знаком, а часть положительна, но имеет неадекватно большую величиную. 
3. В столбце dob_years обнаружен некорректный возраст заёмщика 0 лет, но количество таких значений невелико (101 значение).
4. В столбце children обнаружены некорректные значения признака: -1 и 20.
5. В столбце gender найден один признак с неопознанным значением XNA в количестве 1 штука.
6. В столбцах "уровень образования клиента" и "семейное положение" присутствуют данные заполненные в разном регистре.
7. Столбец цель кредита требует проведения лемматизации.

## Шаг 2. Предобработка данных

### Обработка пропусков

Обработаем пропущенные и нулевые значения, обнаруженные на стадии предварительной обработки.

#### Замена пропущенных значений в столбцах общий трудовой стаж и ежемесячный доход

Проверим связаны ли пропуски в столбцах  days_employed и total_income (находятся ли они в одной строке) и как они связаны с типом занятости.

Определим количество пропущенных значений.

In [12]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Оценим долю пропущенных значений.

In [13]:
df.isna().mean()

children            0.000000
days_employed       0.100999
dob_years           0.000000
education           0.000000
education_id        0.000000
family_status       0.000000
family_status_id    0.000000
gender              0.000000
income_type         0.000000
debt                0.000000
total_income        0.100999
purpose             0.000000
dtype: float64

Определим связаны ли друг с другом пропущенные значения в столбцах days_employed и total_income.

In [15]:
df[(df['days_employed'].isna()) & (df['total_income'].isna())].count()

children            2174
days_employed          0
dob_years           2174
education           2174
education_id        2174
family_status       2174
family_status_id    2174
gender              2174
income_type         2174
debt                2174
total_income           0
purpose             2174
dtype: int64

Определим как связаны пропущенные значения в столбце total_income и типом занятости в столбце income_type.

In [17]:
df[df['total_income'].isna()]['income_type'].value_counts()

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

Результаты показывают, что пропуски в столбцах days_employed и total_income связаны друг с другом и не связаны каким-либо одним типом занятости, а распределены между признаками: сотрудник, компаньон, пенсионер, госслужащий, предприниматель. Вероятно, что появление пропусков связано с техническими причинами и носит случайный характер.

Заменим пропущенные значения на 0, проверим все ли пропущенные значения исправлены.

In [18]:
df = df.fillna(0)
df.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

#### Замена пропущенных значений в столбце ежемесячный доход

Заменим нулевые значения в столбце ежемесячный доход характерными значениями. 

Для этого найдём среднее и медианное значение для всего столбца, а также средние и медианныые значения для типов занятости.

In [19]:
# определим средний ежемесячный доход по всей выборке
total_income_mean = df['total_income'].mean()
print(f'Средний ежемесячный доход по всей выборке: {total_income_mean}')

# определим медианный ежемесячный доход по всей выборке
total_income_median = df['total_income'].median()
print(f'Медианный ежемесячный доход по всей выборке: {total_income_median}')


Средний ежемесячный доход по всей выборке: 150512.84413613725
Медианный ежемесячный доход по всей выборке: 135514.71128002706


Найдём среднее и медианное значения ежемесячного дохода для каждой категории занятости и сравним их друг с другом.

In [20]:
df.groupby('income_type')['total_income'].agg(['mean', 'median']).sort_values(by = 'mean', ascending=False)

,mean,median
income_type,,
предприниматель,249581.572474,249581.572474
компаньон,182195.618704,162401.351555
госслужащий,153679.631678,139034.452386
сотрудник,145342.380477,133546.457238
безработный,131339.751676,131339.751676
пенсионер,122440.317524,110179.690761
студент,98201.625314,98201.625314
в декрете,53829.130729,53829.130729


Из полученных данных видно, что в целом по столбцу ежемесячный доход наблюдается заметное отличие среднего значения от медианного и это говорит о наличии выраженного неравенства в доходах по разным категориям заёмщиков. Данное наблюдение подтверждается и тем, что средние и медианные значения, вычисленные для различных категорий могут отличаться больше, чем в два раза. Например, медианный ежемесячный доход категории пенсионер равен 110179.69, а категории предприниматель - 249581.57. Данные результаты говорят, что для замены нулевых значений лучше использовать медианные значения, вычисленные в зависимости от категории.

Заменим нулевые значения в столбце income_type на медианные в зависимости от категории заёмщика и проверим остались ли нулевые значения.

In [21]:
income_type_analyze = ['сотрудник', 'компаньон', 'пенсионер', 'госслужащий', 'предприниматель']

for item2 in income_type_analyze:
    total_income_item_median2 = df[df['income_type'] == item2]['total_income'].median()
    df.loc[(df['income_type'] == item2) & (df['total_income'] == 0), 'total_income'] = total_income_item_median2
    
df[df['total_income'] == 0]['income_type'].count()

0

#### Замена нулевых значений в столбце общий трудовой стаж

Заменим нулевые значения в столбце  days_employed. 

Предварительный анализ таблицы показал, что в данном столбце содержатся как положительные, так и отрицательные значения, причем отрицательные значения (за исключением их знака) выглядят более реалистично. Предположим, что изначально все данные о трудовом стаже вводились в годах со знаком минус. После этого домножались на количество дней в году, об этом мы можем судить по представленным отрицательным значениям. Положительные значения большой величины могли образоваться, например, при возведении в квадрат количества лет стажа (изначально отрицательного), и последовательного умножения на количество дней в году. Для проверки данной гипотезы разделим положительные значения на 365, возьмем квадратный корень из полученного числа, так мы получим стаж в годах, после чего умножим на 365 и получим исправленный стаж в днях. Данное выражение можно упростить до умножения квадратного корня числа и квадратного корня 365 (равен примерно 19.1).

Произведем вычисления, выведем максимальное и минимальное значения на экран, а также определим в каких категориях находятся данные пропуски.

In [22]:
days_employed_positive = df[df['days_employed'] > 0]['days_employed']
days_employed_positive = (days_employed_positive ** 0.5) * 19.1

display(days_employed_positive.max())
display(days_employed_positive.min())
df[df['days_employed'] > 0]['income_type'].value_counts()

12106.37797391958

10950.959983660534

пенсионер      3443
безработный       2
Name: income_type, dtype: int64

Полученные результаты говорят, что после преобразования, стаж работы находится в интервале от 30 до 33 лет. Учитывая, что эти значения, в основном, принадлежат категории пенсионер, то можно принять выдвинутую выше гипотезу, как верную.

Заменим положительные значения датафрейма на значения, полученные в результате преобразования. Проверим максимальное значение столбца 'days_employed' после замены.

In [23]:
df.loc[df['days_employed'] > 0, 'days_employed'] = days_employed_positive

df['days_employed'].max()

12106.37797391958

Можно видеть, что числа преобразовались.

Возьмем отрицательные значения в столбце days_employed по модулю и проверим минимальное значение столбца после замены.

In [24]:
days_employed_negative = df[df['days_employed'] < 0]['days_employed']
days_employed_negative = abs(days_employed_negative)

df.loc[df['days_employed'] < 0, 'days_employed'] = days_employed_negative
df['days_employed'].min()

0.0

Как можно видеть, отрицательные значения преобразовались.

Выведем на экран средние и медианные значения стажа работы для каждой категории заемщиков.

In [25]:
df.groupby('income_type')['days_employed'].agg(['mean', 'median']).sort_values(by = 'mean', ascending=False)

,mean,median
income_type,,
безработный,11552.632069,11552.632069
пенсионер,10299.135716,11468.045729
в декрете,3296.759962,3296.759962
госслужащий,3057.343890,2385.358043
сотрудник,2095.293025,1360.363902
компаньон,1900.579581,1311.128244
студент,578.751554,578.751554
предприниматель,260.424042,260.424042


Результаты показывают, что длительность трудового стажа значительно зависит от категории заемщика. Например, стаж работы предпринимателя и пенсионера отличается примерно в 40 раз. Также можно видеть, что средние и медианные значения отличаются незначительно. По этим причина заменим нулевые значения в столбце на средние с учетом категории. После замены проверим количество нулевых значений.

In [26]:
income_type_analyze = ['сотрудник', 'компаньон', 'пенсионер', 'госслужащий', 'предприниматель']

for item3 in income_type_analyze:
    total_income_item_mean3 = df[df['income_type'] == item3]['days_employed'].mean()
    df.loc[(df['income_type'] == item3) & (df['days_employed'] == 0), 'days_employed'] = total_income_item_mean3
    
df[df['days_employed'] == 0]['days_employed'].count()

0

#### Обработка нулевых значений в столбце возраст заемщика

Предварительный анализ показал, что в столбце dob_years содержится некорректный возраст заёмщика 0 лет. Определим у каких категорий заемщиков встречается данный признак.

In [28]:
df[df['dob_years'] == 0]['income_type'].value_counts()

сотрудник      55
пенсионер      20
компаньон      20
госслужащий     6
Name: income_type, dtype: int64

Можно видеть, что пропуски носят случайный характер и поскольку данных значений признака пренебрежимо мало (101 значение), то можно удалить строки, содержащие нулевые значения. 

Удалим строки, содержащие нули в столбце dob_years, и обновим индексацию. Проверим остались ли нулевые значения.

In [29]:
df = df[df['dob_years'] != 0]
df = df.reset_index(drop=True)

df[df['dob_years'] == 0]['dob_years'].count()

0

#### Замена некорректых значений в столбцах количество детей и пол заемщика

Заменим некорректные значения в столбцах children и gender наиболее вероятными. Для children - это 0, а для gender - F.

Заменим -1 и 20 в столбце children на 0 и посчитаем уникальные значения.

In [30]:
df.loc[df['children'] == -1, 'children'] = 0
df.loc[df['children'] == 20, 'children'] = 0

df['children'].value_counts()

0    14202
1     4802
2     2042
3      328
4       41
5        9
Name: children, dtype: int64

Заменим XNA в столбце gender на F и посчитаем уникальные значения.

In [31]:
df.loc[df['gender'] == 'XNA', 'gender'] = 'F'

df['gender'].value_counts()

F    14165
M     7259
Name: gender, dtype: int64

Выведем информацию об измененной таблице.

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21424 entries, 0 to 21423
Data columns (total 12 columns):
children            21424 non-null int64
days_employed       21424 non-null float64
dob_years           21424 non-null int64
education           21424 non-null object
education_id        21424 non-null int64
family_status       21424 non-null object
family_status_id    21424 non-null int64
gender              21424 non-null object
income_type         21424 non-null object
debt                21424 non-null int64
total_income        21424 non-null float64
purpose             21424 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

После обработки пропусков и нулевых значений в датафрейме осталось 21424 строк и 12 столбцов. Пропущенные значения в столбце total_income заменены на медианные в зависимости от типа занятости заемщика, а в столбце days_employed на средние с учетом знака значения. Нулевые значения в столбце dob_years удалены вместе со строками.

### Замена типа данных и обработка дубликатов

Перед заменой типа данных проверим наличие явных дубликатов.

In [33]:
df.duplicated().sum()

54

Для удобства дальнейшей категоризации заменим вещественный тип данных в столбцах days_employed и total_income на целочисленный, а данные в столбцах education и family_status приведем к  нижнему регистру.

Заменим вещественный тип данных в столбцах days_employed и total_income на целочисленный. Ход выполнения процесса будем контролировать конструкцией try-except.

In [35]:
try:
    df['days_employed'] = df['days_employed'].astype('int')
    df['total_income'] = df['total_income'].astype('int')
except:
    print('Произошла ошибка, данные не могут быть заменены')

Проверим результат.

In [37]:
df.dtypes

children             int64
days_employed        int64
dob_years            int64
education           object
education_id         int64
family_status       object
family_status_id     int64
gender              object
income_type         object
debt                 int64
total_income         int64
purpose             object
dtype: object

Проверим как изменилось количество дубликатов после замены типа данных

In [38]:
df.duplicated().sum()

54

Изменение вещественного типа чисел на целочисленный не изменило количество дублоикатов.

Приведем данные в столбцах family_status и education к нижнему регистру, после чего проверим получившийся результат.

In [39]:
df['family_status'] = df['family_status'].str.lower()
df['education'] = df['education'].str.lower()

display(df['family_status'].unique())
df['education'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'не женат / не замужем'], dtype=object)

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

Посчитаем количество явных дубликатов после приведения строк к нижнему регистру.

In [40]:
df.duplicated().sum()

71

Изменение регистра помогло выявить скрытые дубликаты. Удалим явные дубликаты и проверим их количество после этого.

In [41]:
df = df.drop_duplicates().reset_index(drop = True)

df.duplicated().sum()

0

**Вывод**

В результате изменения типа данных удалось обнаружить скрытые дубликаты. Все найденные дубликаты были удалены.

### Лемматизация

Проведем лемматизацию целей получения кредита для дальнейшей категоризации данных.

Запишем данные из столбца purpose в строку для лемматизации.

In [42]:
text = ''
for item_lemma in range(len(df['purpose'])):
    text += df.loc[item_lemma, 'purpose']
    text += ' '

Лемматизируем текст столбца "цель покупки" и подсчитаем число упоминаемых слов

In [43]:
from pymystem3 import Mystem
from collections import Counter
m = Mystem()

lemmas = m.lemmatize(text)
print(Counter(lemmas))

Counter({' ': 54787, 'недвижимость': 6328, 'покупка': 5870, 'жилье': 4436, 'автомобиль': 4284, 'образование': 3995, 'с': 2904, 'операция': 2593, 'свадьба': 2310, 'свой': 2223, 'на': 2210, 'строительство': 1873, 'высокий': 1366, 'получение': 1309, 'коммерческий': 1306, 'для': 1286, 'жилой': 1224, 'сделка': 938, 'дополнительный': 902, 'заниматься': 900, 'подержать': 849, 'проведение': 764, 'сыграть': 760, 'сдача': 649, 'семья': 637, 'собственный': 633, 'со': 627, 'ремонт': 605, 'приобретение': 459, 'профильный': 435, 'подержанный': 113, ' \n': 1})


**Вывод**

Лемматизация показала, что самыми часто встречающимися словами в тексте цели кредита являются: недвижимость, покупка, а самыми редко встречающимися - профильный, подержанный. Среди целей кредита можно выделить: приобретение и операции с жилолой или коммерческой недвижимостью, покупку автомобиля, получение образования, проведение свадьбы.

### Категоризация данных

Для ответа на вопросы, поставленные в исследовании, проведем категоризацию данных по семейному положению, наличию детей, уровню дохода и целям кредита.

Семейное положение заемщика уже поделено на категории при формировании датасета. Сосчитаем сколько заемщиков имеют то или иное семейное положение.

In [44]:
df['family_status'].value_counts()

женат / замужем          12290
гражданский брак          4130
не женат / не замужем     2794
в разводе                 1185
вдовец / вдова             954
Name: family_status, dtype: int64

Создадим категорию, определяемую количеством детей children_category. По количеству детей можно разделить заемщиков на категории: "без детей", "с детьми", "многодетный".

Создадим функцию, определяющую категорию в зависимости от количества детей и проверим ее работу.

In [45]:
def children_category_func(quantity):
    if quantity == 0:
        return 'без детей'
    elif 0 < quantity < 3:
        return 'с детьми'
    elif quantity >= 3:
        return 'многодетный'
    else:
        return 'категория не определена'
    
print(children_category_func(-1))
print(children_category_func(0))
print(children_category_func(3))
print(children_category_func(5))

категория не определена
без детей
многодетный
многодетный


Добавим столбец, где хранятся результаты применения функции и проверим результат.

In [46]:
df['children_category'] = df['children'].apply(children_category_func)
df['children_category'].value_counts()

без детей      14144
с детьми        6831
многодетный      378
Name: children_category, dtype: int64

Создадим категорию, определяющую уровень дохода income_category. 

Построим сводную таблицу методом pivot_table(), в которой отобразим как изменяется средняя и медианная величина ежемесячного дохода в зависимости от типа занятости, образования и пола заемщика. отсортируем ее по убыванию среднего дохода.

In [47]:
df.pivot_table(index = ['gender', 'income_type'], columns = 'education', values = 'total_income', 
               aggfunc = ['mean', 'median']).sort_values(by = ['gender', ('mean', 'высшее')], ascending=False)

mean                                     \
education                      высшее      начальное неоконченное высшее   
gender income_type                                                         
M      компаньон        275055.048232  174882.687500       207432.273585   
       предприниматель  249581.000000            NaN                 NaN   
       госслужащий      238927.437500  231274.750000       193715.681818   
       сотрудник        209401.514539  144361.316456       199755.744966   
       пенсионер        185658.128571  122904.150000       157590.600000   
       студент           98201.000000            NaN                 NaN   
       безработный                NaN            NaN                 NaN   
F      предприниматель  499163.000000            NaN                 NaN   
       компаньон        212150.087598  153637.000000       187694.571429   
       безработный      202722.000000            NaN                 NaN   
       госслужащий      173951.310000  114326.250000       150212.466667   
       сотрудник        173795.911227  127988.621212       150535.374429   
       пенсионер        158494.599068  108294.113924       130945.222222   
       в декрете                  NaN            NaN                 NaN   

                                                        median            \
education                     среднее ученая степень    высшее начальное   
gender income_type                                                         
M      компаньон        202126.816901            NaN  217767.5  150100.5   
       предприниматель            NaN            NaN  249581.0       NaN   
       госслужащий      184519.696262            NaN  202310.0  190966.5   
       сотрудник        168183.841654       194310.0  180243.0  133546.0   
       пенсионер        138091.650943        98752.0  150246.0  113017.0   
       студент                    NaN            NaN   98201.0       NaN   
       безработный       59956.000000            NaN       NaN       NaN   
F      предприниматель            NaN            NaN  499163.0       NaN   
       компаньон        162889.583878            NaN  174342.0  148206.5   
       безработный                NaN            NaN  202722.0       NaN   
       госслужащий      142026.580696       111392.0  145060.5  122400.0   
       сотрудник        139345.306599            NaN  144238.5  124397.0   
       пенсионер        127697.809969       255425.0  132524.0  102598.0   
       в декрете         53829.000000            NaN       NaN       NaN   

                                                                     
education              неоконченное высшее   среднее ученая степень  
gender income_type                                                   
M      компаньон                  179370.0  170398.0            NaN  
       предприниматель                 NaN       NaN            NaN  
       госслужащий                166424.0  160213.0            NaN  
       сотрудник                  174615.0  147530.0       198570.0  
       пенсионер                  124667.0  117099.0        98752.0  
       студент                         NaN       NaN            NaN  
       безработный                     NaN   59956.0            NaN  
F      предприниматель                 NaN       NaN            NaN  
       компаньон                  165269.0  154990.5            NaN  
       безработный                     NaN       NaN            NaN  
       госслужащий                139034.0  135072.5       111392.0  
       сотрудник                  133546.0  132162.0            NaN  
       пенсионер                  110940.0  110179.0       255425.0  
       в декрете                       NaN   53829.0            NaN

Из таблицы видно, что заемщики характеризуются высоким неравенством доходов. Величина дохода принимает значения от 53000 до 499163. При этом большая часть заемщиков имеет доход в районе от 100000 до 200000.

Разобъем столбец total_income на несколько частей и посмотрим как изменяется при этом доход.

In [48]:
import statistics

for part_of_income in [0.1, 0.2, 0.25, 0.3, 0.4, 0.5, 0.6, 0.7, 0.75, 0.8, 0.9]:
    income_level = df['total_income'].quantile(part_of_income)
    display(f'{part_of_income * 100} % заемщиков имеют доход ниже {income_level}')

'10.0 % заемщиков имеют доход ниже 78743.8'

'20.0 % заемщиков имеют доход ниже 98553.4'

'25.0 % заемщиков имеют доход ниже 107650.0'

'30.0 % заемщиков имеют доход ниже 112962.4'

'40.0 % заемщиков имеют доход ниже 132162.0'

...

'60.0 % заемщиков имеют доход ниже 161327.8'

'70.0 % заемщиков имеют доход ниже 179782.8'

'75.0 % заемщиков имеют доход ниже 195785.0'

'80.0 % заемщиков имеют доход ниже 214571.60000000003'

'90.0 % заемщиков имеют доход ниже 269886.39999999997'

Можно видеть, что 25 % самых низкообеспеченных заемщиков имеют доход ниже 107650, 25 % самых высокообеспеченных заемщиков выше 195785, а основная масса имеет доход внутри данного диапазона. Полученные значения согласуются со сводной таблицей и могут быть использованы для деления по категориям.

In [49]:
# от этого значения начинается категория со средним доходом
middle_income = df['total_income'].quantile(0.25)
# от этого значения начинается категория с высоким доходом
high_income = df['total_income'].quantile(0.75)

display(middle_income)
high_income

107650.0

195785.0

Создадим функцию, определяющую категорию в зависимости от дохода и проверим ее работу.

In [51]:
def income_category_func(income):
    if income > high_income:
        return 'высокий доход'
    elif middle_income < income <= high_income:
        return 'средний доход'
    elif 0 <= income <= middle_income:
        return 'низкий доход'
    else:
        return 'категория не определена'
    
display(income_category_func(200000))
display(income_category_func(110000))
display(income_category_func(106000))
income_category_func(-1)

'высокий доход'

'средний доход'

'низкий доход'

'категория не определена'

Добавим к датафрейму столбец, в котором хранятся результаты действия функции и проверим результат.

In [52]:
df['income_category'] = df['total_income'].apply(income_category_func)
df['income_category'].value_counts()

средний доход    10676
низкий доход      5339
высокий доход     5338
Name: income_category, dtype: int64

Выделим следующие цели кредитования:

1. Покупка или операции с жилой недвижимостью. Леммы: 'покупка', 'операция', 'жилой', 'свой', 'недвижимость', 'жилье',  'строительство', 'ремонт'.
4. Покупка или операции с коммерческой недвижимостью. Леммы: 'покупка', 'операция', 'коммерческий', 'недвижимость', 'сдача'.
5. Покупка автомобиля. Леммы: 'покупка', 'автомобиль', 'подержанный', 'подержать'.
7. Получение образования. Леммы: 'получение', 'высокий', 'профильный', 'образование', 'дополнительный'.
9. Проведение свадьбы. Леммы: 'свадьба'.

Создадим функцию, определяющую категорию в зависимости от цели кредита и проверим ее работу.

In [53]:
def purpose_category_func(purpose):
    lemmatize_word = m.lemmatize(purpose)
    lemmatize_word
    if 'свадьба' in lemmatize_word:
        return 'сыграть свадьбу'            
    elif 'дополнительный' in lemmatize_word and 'образование' in lemmatize_word:
        return 'получение образования'
    elif 'высокий' in lemmatize_word or 'профильный' in lemmatize_word or 'образование' in lemmatize_word:
        return 'получение образования'
    elif ('подержанный' in lemmatize_word or 'подержать' in lemmatize_word) and 'автомобиль' in lemmatize_word:
        return 'покупка автомобиля'
    elif 'автомобиль' in lemmatize_word:
        return 'покупка автомобиля'
    elif 'ремонт' in lemmatize_word:
        return 'покупка или операции с жилой недвижимостью'
    elif 'строительство' in lemmatize_word:
        return 'покупка или операции с жилой недвижимостью'
    elif 'коммерческий' in lemmatize_word or 'сдача' in lemmatize_word:
        return 'покупка или операции с коммерческой недвижимостью'
    elif 'недвижимость' in lemmatize_word or 'жилье' in lemmatize_word:
        return 'покупка или операции с жилой недвижимостью'
    else:
        return 'категория не определена'
    
display(purpose_category_func('покупка жилья'))
display(purpose_category_func('строительство собственной недвижимости'))
display(purpose_category_func('ремонт жилью'))
display(purpose_category_func('покупка жилья для сдачи'))
display(purpose_category_func('на покупку автомобиля'))
display(purpose_category_func('на покупку подержанного автомобиля'))
display(purpose_category_func('получение образования'))
display(purpose_category_func('дополнительное образование'))
purpose_category_func('свадьба')

'покупка или операции с жилой недвижимостью'

'покупка или операции с жилой недвижимостью'

'покупка или операции с жилой недвижимостью'

'покупка или операции с коммерческой недвижимостью'

'покупка автомобиля'

'покупка автомобиля'

'получение образования'

'получение образования'

'сыграть свадьбу'

Добавим к датафрейму столбец, в котором хранятся результаты действия функции, и проверим результат.

In [54]:
df['purpose_category'] = df['purpose'].apply(purpose_category_func)
df['purpose_category'].value_counts()

покупка или операции с жилой недвижимостью           8809
покупка автомобиля                                   4284
получение образования                                3995
сыграть свадьбу                                      2310
покупка или операции с коммерческой недвижимостью    1955
Name: purpose_category, dtype: int64

**Вывод**

В реультате категоризации было создано 3 новых категории: наличие детей, уровень дохода и цель получения кредита. Категоризацию по семейному положению можно провести на имеющихся данных без формирования нового столбца. Данные показвают, что самыми распространенными категориями среди заемщиков являются женатые (замужние) люди, без детей, со средним доходом и целью кредита - покупка или операции с жилой недвижимостью. Самыми малочисленными категориями: вдовцы, многодетные, с высоким доходом и целью кредита - покупка или операции с коммерческой недвижимостью.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Создадим функцию, которая выводит на экран таблицу, содержащую категорию, общее количество кредитов и вероятность невозврата кредита для каждой категории.

In [55]:
def debt_probability_func(category_type):
    # количество невозвратных кредитов
    df_category_sum = df.groupby([category_type]).agg({'debt' : 'sum'})
    # общее количество кредитов
    df_category_count = df.groupby([category_type]).agg({'debt' : 'count'})
    # вероятность невозврата кредита
    df_category_count['debt_probability'] = round(df_category_sum['debt'] / df_category_count['debt'], 3)
    # отобразим категорию, общее количество кредитов и вероятность невозврата по уменьшению вероятности
    return df_category_count.pivot_table(index = [category_type], values = ('debt_probability', 'debt')).sort_values(by = 'debt_probability', ascending=False)
    #return df_category_count.sort_values(by = 'debt_probability', ascending=False)
    
debt_probability_func('children_category')

,debt,debt_probability
children_category,,
с детьми,6831,0.093
многодетный,378,0.082
без детей,14144,0.075


**Вывод**

Полученные результаты говорят, что самыми надежными заемщиками являются люди без детей (данная категория также берет больше всего кредитов), а самыми ненадежными - люди, имеющие от 1 до 2 детей. Многодетные заемщики являются более надежными по сравнению с теми у кого от 1 до 2 детей, возможно из-за того, что данная категория граждан имеет возможность получать социальную помощь от государства.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Отобразим категорию, общее количество кредитов и вероятность невозврата по уменьшению вероятности.

In [56]:
debt_probability_func('family_status')

,debt,debt_probability
family_status,,
не женат / не замужем,2794,0.098
гражданский брак,4130,0.093
женат / замужем,12290,0.075
в разводе,1185,0.072
вдовец / вдова,954,0.065


**Вывод**

Самыми ненадежными заемщиками являются не женатые, либо живущие в гражданском браке заемщики, их вероятность невозврата кредита приближается к 10%. Самыми надежными - вдовцы и люди в разводе, их вероятность невозврата около 6,5 и 7%, соответственно. Самой распространенной категорией заемщиков являются женатые (замужние) граждане, их надежность находится на среднем уровне и приближается к 7,5%.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Отобразим категорию, общее количество кредитов и вероятность невозврата по уменьшению вероятности.

In [57]:
debt_probability_func('income_category')

,debt,debt_probability
income_category,,
средний доход,10676,0.087
низкий доход,5339,0.080
высокий доход,5338,0.072


**Вывод**

После изменения категоризации по уровню дохода, зависимость между данным уровнем и возвратом кредита в срок сохранилась. Самой надежной категорией заемщиков является категория с высоким уровнем дохода (выше 195785) ее вероятность невозврата около 7 %. Категория заемщиков со средним уровнем дохода (от 107650 до 195785) является самой ненадежной - вероятность невозврата около 9 %. Заемщики с низким уровнем дохода имеет вероятность невозврата (8 %) ниже, чем заемщики со средним.

- Как разные цели кредита влияют на его возврат в срок?

Отобразим категорию, общее количество кредитов и вероятность невозврата по уменьшению вероятности.

In [58]:
debt_probability_func('purpose_category')

,debt,debt_probability
purpose_category,,
покупка автомобиля,4284,0.093
получение образования,3995,0.093
сыграть свадьбу,2310,0.080
покупка или операции с коммерческой недвижимостью,1955,0.077
покупка или операции с жилой недвижимостью,8809,0.071


**Вывод**

Разные цели влияют на вероятность возврата кредита. Цель кредита самых ненадежных заемщиков - это либо покупка автомобиля, либо получение образования. Самые надежные заемщики берут кредит на покупку или операции с жилой недвижимости. Операции с коммерческой недвижимостью немного повышают риск невозврата по сравнению с жилой. Интересно отметить, что большое число заемщиков (больше, чем операций с коммерческой недвижимостью) берет кредит на проведение свадьбы, их надежность находится на среднем уровне (вероятность невозврата 8 %) и выше надежности заемщиков, которые планируют купить автомобиль.

## Шаг 4. Общий вывод

В данной работе проведена предобработка данных, заменены пропущенные, нулевые и некорректные значения, удалены дубликаты, проведена лемматизация и категоризация данных. Оценены зависимости между наличием детей, семейным положением, уровнем дохода, целью кредита и возвратом кредита в срок. Показано, что  самым ненадежным заемщиком будет не женатый, с детьми, со средним доходом , которому нужен кредит на получение образования или покупку автомобиля. А самым надежным - бездетный вдовец с высоким доходом, который берет кредит на покупку или операции с жилой недвижимостью.